# 协程

-可迭代(Iterable):直接作用于for循环的变量
-迭代器(Iterator):不但可以作用于for循环，还可以被next调用
-list是典型的可迭代对象，但不是迭代器
-通过isinstance判断
-iterable和iterator可以转换
-通过iter函数

#学什么，一定要死扣定义。

In [4]:
# 可迭代
l = [ i for i in range(10)]

# l是可迭代的，但不是迭代器
for idx in l:
    print(idx)

# range是个迭代器
for i in range(5):
    print(i)


0
1
2
3
4
5
6
7
8
9
0
1
2
3
4


In [7]:
# isinstance案例
# 判断某个变量是否是一个实例
# 判断是否可迭代
from collections import Iterable
l = [1,2,3,4]
print(isinstance(l,Iterable))

from collections import Iterator
print(isinstance(l,Iterator))

True
False


In [11]:
# iter 函数
s = 'i like python'
print(isinstance(s,Iterable))
print(isinstance(s,Iterator))

s_iter = iter(s)
print(isinstance(s_iter,Iterable))
print(isinstance(s_iter,Iterator))

True
False
True
True


#生成器
1.generator: 一边循环一边计算下一个元素的机制/算法
2.需要满足三个条件：
-每次调用都生产出for循环需要的下一个元素
-如果达到最后一个，爆出StopIteration异常
-可以被next函数调用
3.如何生成一个生成器
-直接使用
-如果函数中包含yield，则这个函数就叫生成器
-next调用函数，遇到yield返回

In [12]:
# 直接使用
l = [x*x for x in range(5)]  # 放在中括号就是列表生成器
print(type(l))

g = (x*x for x in range(5))  # 放在小括号就是生成器
print(type(g))

<class 'list'>
<class 'generator'>


In [13]:
def odd():
    print("step 1")
    print("step 2")
    print("step 3")
    return None

odd()

step 1
step 2
step 3


In [24]:
def odd():
    print("step 1")
    yield 1
    print("step 2")
    yield 2
    print("step 3")
    yield 3
g = odd()  # odd()调用生成器
one = next(g)
print(one)

two = next(g)
print(two)

three = next(g)
print(three)

step 1
1
step 2
2
step 3
3


In [25]:
# 斐波那契额数列的生成器写法
def fib(max):
    n, a, b = 0, 0, 1 # 注意写法
    while n < max:
        yield b
        a,b = b, a+b # 注意写法
        n += 1
        
    #需要注意，爆出异常是的返回值是return的返回值
    return 'Done'

g = fib(5)

for i in range(6):
    rst = next(g)
    print(rst)

1
1
2
3
5


StopIteration: Done

# 协程
- 历史历程
- 3.4引入协程，用yield实现
- 3.5引入协程语法
- 实现的协程比较好的包有asyncio, tornado, gevent
- 定义：协程 是为非抢占式多任务产生子程序的计算机程序组件，协程允许不同入口点在不同位置暂停或开始执行程序”。
- 从技术角度讲，协程就是一个你可以暂停执行的函数，或者干脆把协程理解成生成器
- 协程的实现：
- yield返回
- send调用

In [1]:
# 协程代码案例1
def simple_coroutine():
    print('-> start')
    x = yield
    print('-> recived', x)

#主线程
sc = simple_coroutine()
print(1111)
# 可以使用sc.send(None)，效果一样
next(sc) #预激

print(2222)
sc.send('zhexiao')

1111
-> start
2222
-> recived zhexiao


StopIteration: 

In [28]:
# 案例2，协程的状态
def simple_coroutine(a):
    print('-> start')

    b = yield a
    print('-> recived', a, b)

    c = yield a + b
    print('-> recived', a, b, c)

# runc
sc = simple_coroutine(5)

aa = next(sc)
print(aa)
bb = sc.send(6) # 5, 6
print(bb)
cc = sc.send(7) # 5, 6, 7
print(cc)

-> start
5
-> recived 5 6
11
-> recived 5 6 7


StopIteration: 

In [29]:
# 案例v03
def gen():
    for c in 'AB':
        yield c
# list直接用生成器作为参数
print(list(gen()))

def gen_new():
    yield from 'AB'

print(list(gen_new()))

['A', 'B']
['A', 'B']


In [30]:
# 案例v04， 委派生成器
from collections import namedtuple

'''
解释：
1. 外层 for 循环每次迭代会新建一个 grouper 实例，赋值给 coroutine 变量； grouper 是委派生成器。
2. 调用 next(coroutine)，预激委派生成器 grouper，此时进入 while True 循环，调用子生成器 averager 后，在 yield from 表达式处暂停。
3. 内层 for 循环调用 coroutine.send(value)，直接把值传给子生成器 averager。同时，当前的 grouper 实例（coroutine）在 yield from 表达式处暂停。
4. 内层循环结束后， grouper 实例依旧在 yield from 表达式处暂停，因此， grouper函数定义体中为 results[key] 赋值的语句还没有执行。
5. coroutine.send(None) 终止 averager 子生成器，子生成器抛出 StopIteration 异常并将返回的数据包含在异常对象的value中，yield from 可以直接抓取 StopItration 异常并将异常对象的 value 赋值给 results[key]
'''
ResClass = namedtuple('Res', 'count average')


# 子生成器
def averager():
    total = 0.0
    count = 0
    average = None

    while True:
        term = yield
        # None是哨兵值
        if term is None:
            break
        total += term
        count += 1
        average = total / count

    return ResClass(count, average)


# 委派生成器
def grouper(storages, key):
    while True:
        # 获取averager()返回的值
        storages[key] = yield from averager()


# 客户端代码
def client():
    process_data = {
        'boys_2': [39.0, 40.8, 43.2, 40.8, 43.1, 38.6, 41.4, 40.6, 36.3],
        'boys_1': [1.38, 1.5, 1.32, 1.25, 1.37, 1.48, 1.25, 1.49, 1.46]
    }

    storages = {}
    for k, v in process_data.items():
        # 获得协程
        coroutine = grouper(storages, k)

        # 预激协程
        next(coroutine)

        # 发送数据到协程
        for dt in v:
            coroutine.send(dt)

        # 终止协程
        coroutine.send(None)
    print(storages)

# run
client()

{'boys_2': Res(count=9, average=40.422222222222224), 'boys_1': Res(count=9, average=1.3888888888888888)}


In [31]:
def consumer():
    r = ''
    while True:
        n = yield r
        if not n:
            return
        print('[CONSUMER] Consuming %s...' % n)
        r = '200 OK'

def produce(c):
    c.send(None)
    n = 0
    while n < 5:
        n = n + 1
        print('[PRODUCER] Producing %s...' % n)
        r = c.send(n)
        print('[PRODUCER] Consumer return: %s' % r)
    c.close()

c = consumer()
produce(c)

[PRODUCER] Producing 1...
[CONSUMER] Consuming 1...
[PRODUCER] Consumer return: 200 OK
[PRODUCER] Producing 2...
[CONSUMER] Consuming 2...
[PRODUCER] Consumer return: 200 OK
[PRODUCER] Producing 3...
[CONSUMER] Consuming 3...
[PRODUCER] Consumer return: 200 OK
[PRODUCER] Producing 4...
[CONSUMER] Consuming 4...
[PRODUCER] Consumer return: 200 OK
[PRODUCER] Producing 5...
[CONSUMER] Consuming 5...
[PRODUCER] Consumer return: 200 OK


In [34]:
def averager():
    total = 0.0
    count = 0
    avg = None

    while True:
        num = yield avg
        total += num
        count += 1
        avg = total/count

# run
ag = averager()
# 预激协程
print(next(ag))     # None

print(ag.send(10))  # 10
print(ag.send(20))  # 15

None
10.0
15.0


# 终止协程和异常处理

- 协程中未处理的异常会向上冒泡，传给 next 函数或 send 方法的调用方（即触发协程的对象）。

- ==终止协程的一种方式：发送某个哨符值，让协程退出。内置的 None 和Ellipsis 等常量经常用作哨符值==。

In [1]:
import time
import asyncio
 
now = lambda : time.time()
 
async def do_some_work(x):
    print('Waiting: ', x)
 
start = now()
 
coroutine = do_some_work(2)
 
loop = asyncio.get_event_loop()
loop.run_until_complete(coroutine)
 
print('TIME: ', now() - start)

Waiting:  2
TIME:  0.004000186920166016
